# Implementation

In [1]:
include("src/Misc.jl");
include("src/Datasets.jl");
include("src/BayesianCausality.jl");

In [2]:
using .Misc, .BayesianCausality
import .Datasets

In [3]:
using Distributions
using SpecialFunctions, LinearAlgebra
import Base.Iterators: product
import Random: randperm, seed!

using PyPlot

In [4]:
import Base.Filesystem: mkpath

RESULTS_PATH = "./results/synthetic"
PARAMS_PATH = "./params/synthetic"
DATA_PATH = "./data/synthetic"

"./data/synthetic"

In [5]:
seed!(8457);

## Synthetic Data

In [21]:
function generate_params(A₁,A₂,B₁,B₂)
    num = 0
    for a₁ ∈ A₁, a₂ ∈ A₂, b₁ ∈ B₁, b₂ ∈ B₂
        if a₁ >= b₁ && a₂ >= b₂
                num += 1
                setup = Dict(:name => "params-$num", :P => 250, :Rs => 1:5, :K => 2, 
                             :α => 10.0, :m₁ => 0.0, :m₂ => 0.0, :λ₁ => 0.1, 
                             :λ₂ => 0.1, :a₁ => a₁, :a₂ => a₂, :b₁ => b₁, :b₂ => b₂);
                save_json("$PARAMS_PATH/params-$num.json"; setup...)
        end
    end
    return num
end

generate_params (generic function with 1 method)

In [22]:
function generate_data(T::Int, R::Int; K=2, α=10.0, m₁=0.0, m₂=0.0,
        λ₁=0.1, λ₂=0.1, a₁=1.0, a₂=1.0, b₁=1.0, b₂=1.0, pr=0, rel="^", id=0)
    
    if rel=="^"
        K, a₂, b₂ = 1, a₁, b₁
    end
    
    θ = rand(Dirichlet(R,α))
    ρ₁, ρ₂ = rand(Gamma(a₁,1/b₁),R), rand(Gamma(a₂,1/b₂),R)
    μ₁ = [rand(Normal(m₁, (λ₁ * ρ₁[r])^(-1/2))) for r ∈ 1:R]
    w₂ = [rand(MultivariateNormal(m₂*ones(K), Diagonal(ones(K)/(λ₂*ρ₂[r])))) for r ∈ 1:R]
    
    rs = rand(Categorical(θ),T)
    X₁ = [rand(Normal(μ₁[r], ρ₁[r]^(-1/2))) for r ∈ rs]
    X₂ = [rand(Normal(w₂[r]'*(X₁[t].^(0:K-1)), ρ₂[r]^(-1/2))) for (t,r) ∈ enumerate(rs)]
    
    X = rel=="<-" ? X₂ : X₁
    Y = rel=="<-" ? X₁ : X₂
    
    pair = Dict(:X_label => "X", :Y_label => "Y", :dataset => pr, :weight => 1.0,
                :id => "syth-$id", :relationship => rel, :X => X, :Y => Y)
    save_json("$DATA_PATH/synt-$id.json"; pair...)
    
    return pair
end

generate_data (generic function with 1 method)

In [23]:
T, Rs = 2000, 1:5

(2000, 1:5)

In [28]:
N_PARAMS = generate_params(10.0.^(0:2), 10.0.^(0:2), 10.0.^(0:2), 10.0.^(0:2))

params = [Dict(Symbol(String(p)) => v for (p,v) ∈ load_json("$PARAMS_PATH/params-$n.json"))
                                      for n ∈ 1:N_PARAMS];

In [11]:
N_DATA = 0
for rep ∈ 1:1, R ∈ Rs, pr ∈ 1:N_PARAMS, rel ∈ ["->", "<-", "^"]
    N_DATA += 1
    a₁, a₂, b₁, b₂ = params[pr][:a₁], params[pr][:a₂], params[pr][:b₁], params[pr][:b₂]
    generate_data(T, R; pr=pr, rel=rel, id=N_DATA, a₁=a₁, a₂=a₂, b₁=b₁, b₂=b₂)
end

In [12]:
N₁, N₂ = 1, N_DATA

(1, 540)

In [13]:
synt_data = [Dict(Symbol(String(p)) => v for (p,v) ∈ load_json("$DATA_PATH/synt-$id.json")) 
                                    for id ∈ N₁:N₂];